In [33]:
import numpy as np
import torch
from rosbag import Bag
from tf_bag import BagTfTransformer
from tqdm import tqdm
import argparse
from scipy.interpolate import interp1d
from scipy.spatial.transform import Rotation


class BagTfTransformerWrapper:
    def __init__(self, bag):
        self.tf_listener = BagTfTransformer(bag)
    def waitForTransform(self, parent_frame, child_frame, time, duration):
        return self.tf_listener.waitForTransform(parent_frame, child_frame, time)
    def lookupTransform(self, parent_frame, child_frame, time):
        try:
            return self.tf_listener.lookupTransform(parent_frame, child_frame, time)
        except Exception:
            return (None, None)
        
def extract_imu_data(bag_file, topic, tf_transformer=None, reference_frame=None):
    imu_data = []

    # Update return reference frame
    data_reference_frame = None
    if reference_frame is not None:
        data_reference_frame = reference_frame
    
    transform = None 

    with Bag(bag_file, 'r') as bag:
        for topic_name, msg, t in bag.read_messages(topics=[topic]):
            if reference_frame is not None:

                # Get static tf
                if transform is None:
                    p,q = tf_transformer.lookupTransform(reference_frame,
                                            msg.header.frame_id,
                                            msg.header.stamp)
                    
                    trans = np.array(p)
                    # Convert quaternion to rotation matrix
                    rot = Rotation.from_quat(q).as_matrix()
                    
                    # 1. Rotate angular velocity
                    ang = msg.angular_velocity
                    angular_velocity_target = rot @ np.array([ ang.x, ang.y, ang.z ])
                    
                    # 2. Rotate linear acceleration
                    lin = msg.linear_acceleration
                    linear_acceleration_rotated = rot @ np.array([ lin.x, lin.y, lin.z ])
                    
                    # 3. Account for lever arm effects
                    # 3a. Centripetal acceleration
                    r = np.array(t)
                    centripetal_acc = np.cross(angular_velocity_target, np.cross(angular_velocity_target, trans))
                    
                    # 3b. Tangential acceleration (assuming angular acceleration is zero for simplicity)
                    # If you have angular acceleration data, you can uncomment the following lines:
                    # alpha = ... # Calculate or provide angular acceleration
                    # tangential_acc = np.cross(alpha, r)
                    # linear_acceleration_target = linear_acceleration_rotated + centripetal_acc + tangential_acc
                    
                    # Without angular acceleration:
                    linear_acceleration_target = linear_acceleration_rotated + centripetal_acc
                    
                    msg.linear_acceleration.x = linear_acceleration_target[0]
                    msg.linear_acceleration.y = linear_acceleration_target[1]
                    msg.linear_acceleration.z = linear_acceleration_target[2]
                    msg.angular_velocity.x = angular_velocity_target[0]
                    msg.angular_velocity.y = angular_velocity_target[1]
                    msg.angular_velocity.z = angular_velocity_target[2]


                # Transform imu data to same orign here.
                msg = msg

            ts = t.to_sec()
            if data_reference_frame is None:
                data_reference_frame = msg.header.frame_id

            acc = [msg.linear_acceleration.x, msg.linear_acceleration.y, msg.linear_acceleration.z]
            rot_vel = [msg.angular_velocity.x, msg.angular_velocity.y, msg.angular_velocity.z]
            imu_data.append([ts] + acc + rot_vel)

    return np.array(imu_data, dtype=np.float64), data_reference_frame


import torch
import torch.nn as nn
import torch.optim as optim

def interpolate(t_common, t1_adjusted, y1_adjusted):
    # Ensure inputs are tensors
    t_common = torch.as_tensor(t_common, dtype=torch.float64)
    t1_adjusted = torch.as_tensor(t1_adjusted, dtype=torch.float64)
    y1_adjusted = torch.as_tensor(y1_adjusted, dtype=torch.float64)

    # Find indices for lower bound of each t_common
    indices = torch.searchsorted(t1_adjusted, t_common, right=True) - 1
    
    # Clip indices to valid range
    indices = torch.clamp(indices, 0, len(t1_adjusted) - 2)

    # Get lower and upper bounds
    t0 = t1_adjusted[indices]
    t1 = t1_adjusted[indices + 1]
    y0 = y1_adjusted[indices]
    y1 = y1_adjusted[indices + 1]

    # Compute weights
    w1 = (t_common - t0) / (t1 - t0)
    w0 = 1 - w1

    # Perform linear interpolation
    y_common = w0 * y0 + w1 * y1

    return y_common

class TimeOffsetOptimizer(nn.Module):
    def __init__(self):
        super(TimeOffsetOptimizer, self).__init__()
        self.time_offset = nn.Parameter(torch.tensor(0.0))

    def forward(self, t1, y1, t2, y2):
        """Apply time offset to the second signal."""
        return t1, y1, t2.clone() + self.time_offset, y2


In [31]:
base = "/Data/Projects/GrandTour/lee_k_democracy/2024-09-18-10-59-00/2024-09-18-10-59-00"
imu1_bag = f"{base}_jetson_ap20_0.bag"
imu2_bag =f"{base}_nuc_cpt7_0.bag"     
tf_bag = f"{base}_nuc_tf_0.bag" 
imu1_topic = "/gt_box/ap20/imu"
imu2_topic = "/gt_box/cpt7/imu/data_raw"

imu1_data, reference_frame = extract_imu_data(imu1_bag, imu1_topic)
tf_transformer = BagTfTransformerWrapper(tf_bag)
imu2_data, reference_frame = extract_imu_data(imu2_bag, imu2_topic, tf_transformer = tf_transformer, reference_frame=reference_frame)

t1 = torch.from_numpy( imu1_data[:, 0])
y1 = torch.from_numpy( imu1_data[:, 4])
t2 = torch.from_numpy( imu2_data[:, 0])
y2 = torch.from_numpy( imu2_data[:, 4])

val = max( t1.min(), t2.min() )
t1 = t1 - val
t2 = t2 - val
    

<string>:99: UserWarning: translation should be of type Vector3
<string>:99: UserWarning: rotation should be of type Quaternion


In [32]:
t1,y1,t2,y2 = t1.cuda(), y1.cuda(), t1.cuda(),y1.cuda()
t1 -= 0.1

In [30]:

num_iterations=100000
learning_rate=0.0001

t1,y1,t2,y2 = t1.cuda(), y1.cuda(), t2.cuda(), y2.cuda()
model = TimeOffsetOptimizer()
model.to(t1.device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print("start optimizing")

import matplotlib.pyplot as plt

for j in range(num_iterations):
    optimizer.zero_grad()
    
    # Apply time offset
    t1_adjusted, y1_adjusted, t2_adjusted, y2_adjusted = model(t1, y1, t2, y2)
    
    # Interpolate both signals to a common time grid
    t_min = min(t1_adjusted.min(), t2_adjusted.min())
    t_max = max(t1_adjusted.max(), t2_adjusted.max())
    t_common = torch.linspace(t_min, t_max, 100000)
    t_common = t_common.to(t1.device)
    
    y1_interp = interpolate(t_common, t1_adjusted, y1_adjusted)
    y2_interp = interpolate(t_common, t2_adjusted, y2_adjusted)
    
    # Compute MSE loss
    loss = nn.MSELoss()(y1_interp, y2_interp)
    
    # Backpropagate and update
    loss.backward()
    optimizer.step()
    print(f'{j} Time Offset{model.time_offset.item()}, loss {loss.item()}')
    if j % 1000 == 0:
        plt.plot(t_common.cpu().detach().numpy(), y1_interp.cpu().detach().numpy(), label='y1_interp')
        plt.plot(t_common.cpu().detach().numpy(), y2_interp.cpu().detach().numpy(), label='y2_interp')
        plt.legend()
        plt.xlabel('t_common')
        plt.ylabel('Values')
        plt.title(f'{j} Time Offset{model.time_offset.item()}, loss {loss.item()}')
        plt.show()


start optimizing
0 Time Offset-9.999930625781417e-05, loss 0.018494156814086907
1 Time Offset-9.087446960620582e-05, loss 0.01849495793656534
2 Time Offset-4.78678266517818e-05, loss 0.018494788767021056
3 Time Offset-1.4100521184445824e-05, loss 0.018494567835977617
4 Time Offset-1.9411774701438844e-05, loss 0.018494135091355543
5 Time Offset-3.113286948064342e-05, loss 0.018494343505072956
6 Time Offset-3.835794996120967e-05, loss 0.018494456130531006
7 Time Offset-4.3614832975436e-05, loss 0.018494457268613915
8 Time Offset-6.0236649005673826e-05, loss 0.018494454086795883
9 Time Offset-7.715508399996907e-05, loss 0.0184945694183025
10 Time Offset-7.192813791334629e-05, loss 0.018494765260626905
11 Time Offset-5.368712663766928e-05, loss 0.01849462206379072
12 Time Offset-3.3400760003132746e-05, loss 0.01849456719021076
13 Time Offset-1.4486844520433806e-05, loss 0.018494456607565015
14 Time Offset-1.7866299458546564e-05, loss 0.01849413446064384
15 Time Offset-2.695417424547486e-05

KeyboardInterrupt: 